<a href="https://colab.research.google.com/github/hhicks13/portfolio-optimization/blob/main/Lagrange_Vault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


\begin{array}{ll} \mbox{max} & y^T\mathbf{u} - \frac{1}{2} \gamma y^T M y\\
\mbox{subject to} & {\bf \log y}^Tp = -F,
\end{array}
$\textbf{Solution:}$
> Define the Lagrangian:
$$\mathcal{L}(y,\lambda) = y^T\mathbf{u} - \frac{1}{2}\gamma y^T M y - \lambda(y^Tp + F) $$
> Derive 1st order conditions:
$$\begin{align} \frac{\partial\mathcal{L}}{\partial y} &=& \mathbf{u} -\gamma M y - \gamma p  \overset{\Delta}{=} 0 \\
 \frac{\partial\mathcal{L}}{\partial \lambda} &=& F + y^T {\bf p} \overset{\Delta}{=} 0
 \end{align}$$


$\textbf{Vault Optimization via Lagrangian }$ \
$\textbf{ 1 }$ Let $I_n$ be the identity matrix, and $D$ be the initial deposit.We define the following terms:
$$\begin{align}
\mathbf{u} &\overset{\Delta}{=}& n^{-1}\mathbf{1}_n \\
D &\overset{\Delta}{=}& \texttt{ initial deposit} \\
C_i &\overset{\Delta}{=}& \texttt{ pool i C value} \\
p_i &\overset{\Delta}{=}& \texttt{ pool i free liquidity} \\
w_i &\overset{\Delta}{=}& \texttt{pool i weights for deposit} \\
x_i &\overset{\Delta}{=}& e^{-\frac{w_i D}{p_i}}  \\
y_i &\overset{\Delta}{=}& \log (C_ix_i) \\
&=& \log C_i + \log x_i
\end{align}$$

$\textbf{ 2 }$Matrix Formulation of the log-normal mean estimator $y^T\mathbf{u}$: \
$$
\begin{align}
y^T\mathbf{u} &=& \overline{\log Cx} \\
&=& \frac{\sum_i \log(Cx)_i}{n}\\
\end{align}
$$ \
$\textbf{ 3 }$Matrix Formulation of the log-normal sample covariance estimator $y^TMy$: \
$$\begin{align}
M &=& {I}_n - n^{-1}\mathbf{1}_n\mathbf{1}_n^T \\
y^TMy &=& (n - 1)\sum_i^n({\log(Cx)}_i - \overline{\log (Cx)})^2 \\
\end{align}
$$ \
$\textbf{ 4 }$Due to the fact that all deposits cause a decrease in C value, and all weights must be non-negative and sum to 1, we have the following conditions that must hold:
$$\begin{align}
 -\ln x_i \frac{p_i}{D} &=& w_i \\
 \sum p_i \ln x_i + D &=& 0
 \end{align}$$ \
$\textbf{ 5 }$Proof that condition from $\mathbf{4}$: $\sum_i p_i \log x_i + D = 0$ is equivalent to condition $\sum_i p_i y_i + F = 0 $ \
$$\begin{align}
y &=& \log Cx \\
&=& \log C + \log x \\
y^Tp &=& \sum_i p_i(\log C_i + \log x_i) \\
&=& \log C^T p + \log x^T p \\
F &\overset{\Delta}{=}& D - \log C^T p\\
y^Tp + F &=& \log x^T p + D \\
\end{align}$$





In [2]:
import numpy as np

def u(n):
  return (1/n)*np.ones(n)

def M(n):
  return np.identity(n) + (-1/n)*np.ones(shape=(n,n))

def F(log_c,p,D):
  resid = log_c.T@p
  return D - resid

def expression(y,p,log_c,D):
  _F = F(log_c,p,D)
  return y.T@p + _F

def weights(y,p,log_c,D):
  wdp =  -1*(y - log_c)
  w = cp.multiply(wdp,p)/D 
  return w

def extract_w(y,p,log_c,D):
  wdp = -1*(y-log_c)
  w = np.multiply(wdp,p)/D
  return w

In [3]:
def solve_portfolio(n,_c,_p,D):
  _u = u(n)
  _M = M(n)
  y = cp.Variable(n)
  _log_c = np.log(_c)
  gamma = cp.Parameter(nonneg=True)
  mean = _u.T@y 
  var = cp.quad_form(y, _M)
  prob = cp.Problem(cp.Maximize(mean - gamma*var), 
                  [expression(y,_p,_log_c,D) == 0,
                   cp.sum(weights(y,_p,_log_c,D)) == 1,
                   weights(y,_p,_log_c,D) >= 0],)
  gamma.value = 1.0
  prob.solve()
  return extract_w(y.value,_p,_log_c,D)

In [4]:
import numpy as np
import cvxpy as cp
"""
corner case
"""
n = 1
_c = np.ones(n)*10
_p = np.ones(n)*10
print("corner case 1:")
print("C:",_c,"\n")
print("P:",_p,"\n")

D = _p[0]
cutoff = 4
w0 = np.around(solve_portfolio(n,_c,_p,D),cutoff)
print("D = p \n Weights: ",w0,"\n sum(weights):",sum(w0),"\n")

corner case 1:
C: [10.] 

P: [10.] 

D = p 
 Weights:  [1.] 
 sum(weights): 1.0 



In [5]:
n = 3
_c = np.array([1,1000,1])
_p = np.array([10,10,1000])
print("corner case 2")
print("C:",_c,"\n")
print("P:",_p,"\n")

D = 60
cutoff = 4
w0 = np.around(solve_portfolio(n,_c,_p,D),cutoff)
print("D = p \n Weights: ",w0,"\n sum(weights):",sum(w0),"\n")

corner case 2
C: [   1 1000    1] 

P: [  10   10 1000] 

D = p 
 Weights:  [-0.  1. -0.] 
 sum(weights): 1.0 



# case 0 #
equal c
equal p

In [6]:
case0 = "equal c,equal p"
n = 6
_c = np.ones(n)*10
_p = np.ones(n)*10
print(case0)
print("C:",_c,"\n")
print("P:",_p,"\n")

D1 = np.mean(_p)
D2 = np.sum(_p)
D3 = np.sum(_p)/10
cutoff = 3
w0 = np.around(solve_portfolio(n,_c,_p,D1),cutoff)
print("D = mean(p)\n Weights: ",w0,"\n sum(weights):",sum(w0),"\n")
w1 = np.around(solve_portfolio(n,_c,_p,D2),cutoff)
print("D = sum(p)\n Weights: ",w0,"\n sum(weights):",sum(w1),"\n")
w2 = np.around(solve_portfolio(n,_c,_p,D3),cutoff)
print("D = sum(p)/10 \n Weights: ",w0,"\n sum(weights):",sum(w1),"\n")

equal c,equal p
C: [10. 10. 10. 10. 10. 10.] 

P: [10. 10. 10. 10. 10. 10.] 

D = mean(p)
 Weights:  [0.167 0.167 0.167 0.167 0.167 0.167] 
 sum(weights): 1.002 

D = sum(p)
 Weights:  [0.167 0.167 0.167 0.167 0.167 0.167] 
 sum(weights): 1.002 

D = sum(p)/10 
 Weights:  [0.167 0.167 0.167 0.167 0.167 0.167] 
 sum(weights): 1.002 



# Case 1 #
equal c,
unequal p

In [7]:
case1 = "equal c,unequal p"
n = 6
_c = np.ones(n)*10
_p = np.arange(1,n+1)*200
D1 = np.mean(_p)
D2 = np.sum(_p)
D3 = np.sum(_p)/10
cutoff = 3
print(case1)
print("C:",_c,"\n")
print("P:",_p,"\n")

w0 = np.around(solve_portfolio(n,_c,_p,D1),cutoff)
print("D = mean(p)\n Weights: ",w0,"\n sum(weights):",sum(w0),"\n")
w1 = np.around(solve_portfolio(n,_c,_p,D2),cutoff)
print("D = sum(p)\n Weights: ",w1,"\n sum(weights):",sum(w1),"\n")
w2 = np.around(solve_portfolio(n,_c,_p,D3),cutoff)
print("D = sum(p)/10 \n Weights: ",w2,"\n sum(weights):",sum(w1),"\n")

equal c,unequal p
C: [10. 10. 10. 10. 10. 10.] 

P: [ 200  400  600  800 1000 1200] 

D = mean(p)
 Weights:  [0.025 0.063 0.116 0.181 0.261 0.354] 
 sum(weights): 1.0 

D = sum(p)
 Weights:  [0.044 0.09  0.138 0.189 0.242 0.297] 
 sum(weights): 1.0 

D = sum(p)/10 
 Weights:  [0.012 0.043 0.098 0.175 0.275 0.398] 
 sum(weights): 1.0 



# Case 2 #
unequal c, unequal p

In [8]:
case2 =" unequal c unequal p "
n = 7
_c = np.linspace(0.9,2*np.exp(1),n)
_p = np.arange(1,n+1)*200
D1 = np.mean(_p)
D2 = np.sum(_p)
D3 = np.sum(_p)/10
cutoff = 3
print(case2)
print("C:",_c,"\n")
print("P:",_p,"\n")

w0 = np.around(solve_portfolio(n,_c,_p,D1),cutoff)
print("D = mean(p)\n Weights: ",w0,"\n sum(weights):",sum(w0),"\n")
w1 = np.around(solve_portfolio(n,_c,_p,D2),cutoff)
print("D = sum(p)\n Weights: ",w1,"\n sum(weights):",sum(w1),"\n")
w2 = np.around(solve_portfolio(n,_c,_p,D3),cutoff)
print("D = sum(p)/10 \n Weights: ",w2,"\n sum(weights):",sum(w1),"\n")

 unequal c unequal p 
C: [0.9        1.65609394 2.41218789 3.16828183 3.92437577 4.68046971
 5.43656366] 

P: [ 200  400  600  800 1000 1200 1400] 

D = mean(p)
 Weights:  [-0.    -0.    -0.    -0.     0.124  0.329  0.547] 
 sum(weights): 1.0 

D = sum(p)
 Weights:  [0.    0.014 0.061 0.121 0.19  0.266 0.349] 
 sum(weights): 1.001 

D = sum(p)/10 
 Weights:  [-0.     0.    -0.    -0.     0.071  0.328  0.601] 
 sum(weights): 1.001 

